# Training

In [29]:
# imports
import os
import pathlib
import torch

import albumentations as A
import numpy as np
from pytorch_lightning import Trainer
from pytorch_lightning import seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor, EarlyStopping
from pytorch_lightning.loggers.neptune import NeptuneLogger
from torch.utils.data import DataLoader

from scripts.datasets import ObjectDetectionDataSet
from scripts.faster_RCNN import FasterRCNN_lightning
from scripts.faster_RCNN import get_fasterRCNN_resnet
from scripts.transformations import Clip, ComposeDouble
from scripts.transformations import AlbumentationWrapper
from scripts.transformations import FunctionWrapperDouble
from scripts.transformations import normalize_01
from scripts.utils import get_filenames_of_path, collate_double
from scripts.utils import log_mapping_neptune
from scripts.utils import log_model_neptune
from scripts.utils import log_packages_neptune

In [26]:
# hyper-parameters
params = {'BATCH_SIZE': 2,
          'SAVE_DIR': None,  # checkpoints will be saved to cwd
          'GPU': 1,  # set to None for cpu training
          'LR': 0.001,
          'PRECISION': 32,
          'SEED': 42,
          'PROJECT': 'Heads',
          'EXPERIMENT': 'heads',
          'MAXEPOCHS': 500,
          'BACKBONE': 'resnet34',
          'FPN': False,
          'ANCHOR_SIZE': ((32, 64, 128, 256, 512),),
          'ASPECT_RATIOS': ((0.5, 1.0, 2.0),),
          'MIN_SIZE': 1024,
          'MAX_SIZE': 1024,
          'IMG_MEAN': [0.485, 0.456, 0.406],
          'IMG_STD': [0.229, 0.224, 0.225],
          'IOU_THRESHOLD': 0.5,
          'DATASET_SIZE': 20,
          'CLASSES':['bird','nest'],
          }

In [8]:
# save directory
if not params['SAVE_DIR']:
    save_dir = os.getcwd()

In [9]:
# root directory
root = pathlib.Path('Entrainement')

In [10]:
# input and target files
inputs = get_filenames_of_path(root / 'Input')
targets = get_filenames_of_path(root / 'Target')

inputs.sort()
targets.sort()


AssertionError: No files found in path: Entrainement/Input

In [11]:
# mapping
mapping = {}
for i in range(len(params['CLASSES'])):
    mapping = {
        params['CLASSES'][i]: i+1,
    }

In [13]:
# training transformations and augmentations
transforms_training = ComposeDouble([
    Clip(),
    AlbumentationWrapper(albumentation=A.HorizontalFlip(p=0.5)),
    AlbumentationWrapper(albumentation=A.VerticalFlip(p=0.5)),
    AlbumentationWrapper(albumentation=A.RandomFog(fog_coef_lower=0, fog_coef_upper=0.5, alpha_coef=0.001, always_apply=False, p=0.3)),
    AlbumentationWrapper(albumentation=A.GaussianBlur(blur_limit=(3, 7), p=0.3)),
    AlbumentationWrapper(albumentation=A.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2,0.2), brightness_by_max=True, p=0.3)),

    FunctionWrapperDouble(np.moveaxis, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01)
])

In [14]:
# validation transformations
transforms_validation = ComposeDouble([
    Clip(),
    FunctionWrapperDouble(np.moveaxis, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01)
])

In [15]:
# test transformations
transforms_test = ComposeDouble([
    Clip(),
    FunctionWrapperDouble(np.moveaxis, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01)
])

In [16]:
# random seed
seed_everything(params['SEED'])

Global seed set to 42


42

In [17]:
Dataset_Size = params['DATASET_SIZE']
TSet_Size = Dataset_Size - Dataset_Size//5
VSet_Size = Dataset_Size//10

In [12]:
# training validation test split
inputs_train, inputs_valid, inputs_test = inputs[:TSet_Size], inputs[TSet_Size:TSet_Size+VSet_Size], inputs[TSet_Size+VSet_Size:]
targets_train, targets_valid, targets_test = targets[:TSet_Size], targets[TSet_Size:TSet_Size+VSet_Size], targets[TSet_Size+VSet_Size:]

In [13]:
# dataset training
dataset_train = ObjectDetectionDataSet(inputs=inputs_train,
                                       targets=targets_train,
                                       transform=transforms_training,
                                       use_cache=True,
                                       convert_to_format=None,
                                       mapping=mapping)

In [14]:
# dataset validation
dataset_valid = ObjectDetectionDataSet(inputs=inputs_valid,
                                       targets=targets_valid,
                                       transform=transforms_validation,
                                       use_cache=True,
                                       convert_to_format=None,
                                       mapping=mapping)

In [15]:
# dataset test
dataset_test = ObjectDetectionDataSet(inputs=inputs_test,
                                      targets=targets_test,
                                      transform=transforms_test,
                                      use_cache=True,
                                      convert_to_format=None,
                                      mapping=mapping)

In [16]:
# dataloader training
dataloader_train = DataLoader(dataset=dataset_train,
                              batch_size=params['BATCH_SIZE'],
                              shuffle=True,
                              num_workers=0,
                              collate_fn=collate_double)

In [17]:
# dataloader validation
dataloader_valid = DataLoader(dataset=dataset_valid,
                              batch_size=1,
                              shuffle=False,
                              num_workers=0,
                              collate_fn=collate_double)

In [18]:
# dataloader test
dataloader_test = DataLoader(dataset=dataset_test,
                             batch_size=1,
                             shuffle=False,
                             num_workers=0,
                             collate_fn=collate_double)

In [27]:
# model init
model = get_fasterRCNN_resnet(num_classes=len(params['CLASSES']),
                              backbone_name=params['BACKBONE'],
                              anchor_size=params['ANCHOR_SIZE'],
                              aspect_ratios=params['ASPECT_RATIOS'],
                              fpn=params['FPN'],
                              min_size=params['MIN_SIZE'],
                              max_size=params['MAX_SIZE'])

In [21]:
# lightning init
task = FasterRCNN_lightning(model=model, lr=params['LR'], iou_threshold=params['IOU_THRESHOLD'])

In [22]:
# callbacks
checkpoint_callback = ModelCheckpoint(monitor='Validation_mAP', mode='max')
learningrate_callback = LearningRateMonitor(logging_interval='step', log_momentum=False)
early_stopping_callback = EarlyStopping(monitor='Validation_mAP', patience=50, mode='max')

In [23]:
# trainer init
trainer = Trainer(gpus=params['GPU'],
                  precision=params['PRECISION'],  # try 16 with enable_pl_optimizer=False
                  callbacks=[checkpoint_callback, learningrate_callback, early_stopping_callback],
                  default_root_dir=params['SAVE_DIR'],  # where checkpoints are saved to
                  num_sanity_val_steps=0,
                  )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [24]:
# start training
trainer.max_epochs = params['MAXEPOCHS']
trainer.fit(task,
            train_dataloader=dataloader_train,
            val_dataloaders=dataloader_valid)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | FasterRCNN | 50.4 M
-------------------------------------
50.4 M    Trainable params
0         Non-trainable params
50.4 M    Total params
201.736   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [25]:
# start testing
trainer.test(ckpt_path='best', test_dataloaders=dataloader_test)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'Test_AP_1': 0.9624457970322631, 'Test_mAP': 0.9624457970322631}
--------------------------------------------------------------------------------


[{'Test_mAP': 0.9624457970322631, 'Test_AP_1': 0.9624457970322631}]

In [32]:
torch.save(model.state_dict(), 'Entrainement/Models/best_model.pt')